In [1]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import gmean, ttest_ind
from statsmodels.stats.multitest import multipletests
import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.stats import f_oneway
import scipy.stats as ss
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.api as sm


# transcriptome
from statsmodels.formula.api import ols

In [3]:
proteome = pd.read_excel("/Users/mortezaabyadeh/Documents/npc-fty/published papers/human data.xlsx")

In [4]:
proteome.head()

,Gene names,log2 LFQ CTR_1,log2 LFQ CTR_2,log2 LFQ CTR_3,log2 LFQ NPC_1,log2 LFQ NPC_2,log2 LFQ NPC_3,log2 LFQ NPC_5,log2 LFQ NPC_6,log2 LFQ NPC_7,log2 LFQ NPC_8
0,RPLP2,32.184105,32.034897,32.297783,31.448992,31.509327,31.600187,31.682570,31.991657,31.687666,31.464802
1,GNB4,29.801868,29.878445,29.542280,30.133692,30.154966,30.420113,30.123615,30.166386,30.004772,30.390385
2,STAM2,25.947092,26.186861,26.162579,25.631037,25.365263,25.660061,NaN,NaN,25.587378,NaN
3,PDLIM5,28.913250,29.047035,29.341871,28.806427,28.674215,28.289604,28.659046,28.724262,28.560671,28.269770
4,ALYREF,30.091755,30.157408,30.239567,29.776901,29.981825,29.744659,29.906700,30.057240,29.671518,29.874245


In [5]:
proteome.columns

Index(['Gene names', 'log2 LFQ CTR_1', 'log2 LFQ CTR_2', 'log2 LFQ CTR_3',
       'log2 LFQ NPC_1', 'log2 LFQ NPC_2', 'log2 LFQ NPC_3', 'log2 LFQ NPC_5',
       'log2 LFQ NPC_6', 'log2 LFQ NPC_7', 'log2 LFQ NPC_8'],
      dtype='object')

In [8]:
control_cols = ['log2 LFQ CTR_1', 'log2 LFQ CTR_2', 'log2 LFQ CTR_3']
npc_cols = ['log2 LFQ NPC_1', 'log2 LFQ NPC_2', 'log2 LFQ NPC_3', 'log2 LFQ NPC_5','log2 LFQ NPC_6', 'log2 LFQ NPC_7', 'log2 LFQ NPC_8']

In [11]:
def filter_rows(row):
    control_nan_count = row[control_cols].isna().sum()
    npc_nan_count = row[npc_cols].isna().sum()
    return not ((control_nan_count > 2) or (npc_nan_count > 2)) 

filtered_proteome = proteome[proteome.apply(filter_rows, axis=1)]

In [12]:
print(proteome.shape)
proteome.isna().sum()

(3358, 11)


Gene names          2
log2 LFQ CTR_1      0
log2 LFQ CTR_2      0
log2 LFQ CTR_3      0
log2 LFQ NPC_1     84
log2 LFQ NPC_2     26
log2 LFQ NPC_3    166
log2 LFQ NPC_5    258
log2 LFQ NPC_6    233
log2 LFQ NPC_7    287
log2 LFQ NPC_8    337
dtype: int64

In [14]:
print(filtered_proteome.shape)
filtered_proteome.isna().sum()

(3162, 11)


Gene names          2
log2 LFQ CTR_1      0
log2 LFQ CTR_2      0
log2 LFQ CTR_3      0
log2 LFQ NPC_1     44
log2 LFQ NPC_2      4
log2 LFQ NPC_3     68
log2 LFQ NPC_5    137
log2 LFQ NPC_6    120
log2 LFQ NPC_7    156
log2 LFQ NPC_8    199
dtype: int64

In [15]:
def impute_group(group):
    imputer = IterativeImputer(estimator=RandomForestRegressor(), max_iter=10, random_state=0)
    imputed_group = imputer.fit_transform(group)
    return pd.DataFrame(imputed_group, columns=group.columns, index=group.index)

control_imputed = impute_group(filtered_proteome[control_cols])
filtered_proteome[control_cols] = control_imputed

npc_imputed = impute_group(filtered_proteome[npc_cols])
filtered_proteome[npc_cols] = npc_imputed


In [16]:
print(filtered_proteome.isna().sum())
print(filtered_proteome.shape)

Gene names        2
log2 LFQ CTR_1    0
log2 LFQ CTR_2    0
log2 LFQ CTR_3    0
log2 LFQ NPC_1    0
log2 LFQ NPC_2    0
log2 LFQ NPC_3    0
log2 LFQ NPC_5    0
log2 LFQ NPC_6    0
log2 LFQ NPC_7    0
log2 LFQ NPC_8    0
dtype: int64
(3162, 11)


In [17]:
filtered_proteome.to_excel("/Users/mortezaabyadeh/Documents/npc-fty/published papers/imputedhumandata.xlsx")